# Atualizações

* EDA
* Criar novas colunas
* Criar Função que treine os modelos XGBosst e Random Forest e gere uma tabela com as metricas de ambos
* Criar Função que função que treine o modelo escolhido e atualize a tabela de score das versões
* Escolher outros valores para os parametros no fine tuning
* Criar uma função que faça todas as transformações do dataset test.csv

# 0.0 Imports

In [113]:
import inflection
import io
import math
import time
import random
import numpy as np
import pandas as pd
import seaborn as snb
import time
import sweetviz as sv
import warnings

from xgboost import XGBClassifier
from scipy import stats
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import Lasso, LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn import tree
from boruta import BorutaPy
from sklearn.preprocessing import RobustScaler
from category_encoders import CountEncoder, TargetEncoder
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import LabelEncoder 
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
import datetime
from sklearn.naive_bayes import BernoulliNB



%matplotlib inline
snb.set_style('whitegrid')
warnings.filterwarnings('ignore')


## Functions

In [149]:
def show_dtypes(df):
    print(df.dtypes)
    return None

def show_dimensions(df):
    print('Number of rows: {}' .format(df.shape[0]))
    print('Number of columns: {}' .format(df.shape[1]))
    return None

def cramer_v( x, y ):
    cm = pd.crosstab( x, y ).to_numpy()
    n = cm.sum()
    r, k = cm.shape
    
    chi2 = stats.chi2_contingency( cm )[0]
    chi2corr = max( 0, chi2 - (k-1)*(r-1)/(n-1) )
    
    kcorr = k - (k-1)**2/(n-1)
    rcorr = r - (r-1)**2/(n-1)
    return round(np.sqrt( (chi2corr/n) / ( min( kcorr-1, rcorr-1 ) ) ),2)

def train_models(x_train, x_test, y_train, y_test):

    # Treinando modelo Random Forest
    rf = RandomForestClassifier(random_state=42)
    rf.fit(x_train, y_train)
    rf_f1_score = f1_score(y_test, rf.predict(x_test))

    # Treinando modelo XGBoost
    xgb = XGBClassifier(random_state=42)
    xgb.fit(x_train, y_train)
    xgb_f1_score = f1_score(y_test, xgb.predict(x_test))

    # Criando a tabela com o nome do algoritmo e o f1_score
    table = pd.DataFrame({'Algoritmo': ['Random Forest', 'XGBoost'], 'F1 Score': [
                         rf_f1_score, xgb_f1_score]})

    return table

def transfomation_test(model, dft, listID, name = 'submission'):
    # Rename 
    dft = dft.rename(columns={'id':'id','Classificação do hotel': 'hotel_rating', 'Meses da reserva até o check-in': 'month_booking',
                            'Número de pernoites reservadas': 'number_stays_booking', 'Número de hospedes': 'number_guests',
                            'Regime de alimentação': 'meal_regime', 'Nacionalidade': 'nationality', 'Forma de Reserva': 'reservation_Form',
                            'Já se hospedou anterioremente': 'previously_hosted', 'Tipo do quarto reservado': 'type_booked_room',
                            'Reserva feita por agência de turismo': 'tourism_agency_booking', 'Reserva feita por empresa': 'company_booking',
                            'Reserva com Estacionamento': 'parking_Reservation', 'Reserva com Observações': 'reservation_Observations', 'Reserva Cancelada': 'booking_canceled'})



    ## Fill NA
    dft['nationality'] = dft['nationality'].apply(
        lambda x: 'Spain' if pd.isnull(x) else x)
    # dft['number_guests'] = dft['number_guests'].apply(
    #     lambda x: 1 if pd.isnull(x) else x)
    # dft['number_guests'] = dft['number_guests'].astype('int64')
    ## Change type
    dft['id'] = dft['id'].astype('int64')

    ## Rescaling
    rs = RobustScaler()

    # meses_da_reserva_ate_o_check-in - Robust Scaler
    dft['id'] = rs.fit_transform( dft[['id']].values )

    # meses_da_reserva_ate_o_check-in - Robust Scaler
    dft['month_booking'] = rs.fit_transform(dft[['month_booking']].values)

    # numero_de_pernoites_reservadas - Robust Scaler
    dft['number_stays_booking'] = rs.fit_transform( dft[['number_stays_booking']].values )

    # numero_de_hospedes - Robust Scaler
    dft['number_guests'] = rs.fit_transform( dft[['number_guests']].values )
    ## Transformation
    # Label Encoder
    le = LabelEncoder()
    label_cols = ['meal_regime', 'nationality', 'reservation_Form',
                'type_booked_room', 'reservation_Observations']
    for col in label_cols:
        dft[col] = le.fit_transform(dft[col])

    # Ordinal Encoding
    classificacao_dict = {'5 estrelas': 5, '4 estrelas': 4}
    dft['hotel_rating'] = dft['hotel_rating'].map(classificacao_dict)

    # Boolean
    bool_cols = ['previously_hosted', 'tourism_agency_booking',
                'company_booking', 'parking_Reservation']
    for col in bool_cols:
        dft[col] = dft[col].apply(
            lambda x: 0 if x == 'Não' else 1 if x == 'Sim' else x)

    # transform into int
    dft['hotel_rating'] = dft['hotel_rating'].astype('int')
    dft['previously_hosted'] = dft['previously_hosted'].astype('int')
    dft['tourism_agency_booking'] = dft['tourism_agency_booking'].astype('int')
    dft['company_booking'] = dft['company_booking'].astype('int')
    dft['parking_Reservation'] = dft['parking_Reservation'].astype('int')
    
    ## Predict
    cols = ['id','hotel_rating','month_booking','nationality','parking_Reservation', 'reservation_Observations', 'number_stays_booking', 'reservation_Form']
    y_pred_test = model.predict(dft[cols].values)

    y_pred_test = model.predict(dft.values)

    submission = pd.concat([listID, pd.Series(y_pred_test, name='Reserva Cancelada')], axis=1)

    ## Create submission file
    submission['Reserva Cancelada'] = submission['Reserva Cancelada'].astype(int)

    submission.to_csv('../hachday_hotel-chain/submission/'+ name + '.csv', index=False)
    return




# 1.0 Loading Data

In [115]:
X = pd.read_csv('../hachday_hotel-chain/dataset/X.csv')
y = pd.read_csv('../hachday_hotel-chain/dataset/y.csv')

In [116]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 42)

#  Machine Learning Model

In [129]:
import sklearn


In [133]:

# tabela = train_models( X_train, X_test, y_train, y_test )
# print(tabela)


# Hyperparameter Fine Tuning


In [134]:
t_start = time.time()
print('Start:',datetime.datetime.now().strftime('%d-%m-%Y %H:%M:%S'))

Start: 09-05-2023 16:33:17


In [79]:
param = {'learning_rate': [0.05,0.1],
         'max_depth': [5,10],
         'n_estimators': [500, 1000],
         'subsample': [0.4,0.8]}
# Calculate weights

# y array of weights
sample_weights = compute_sample_weight(class_weight='balanced', y=y)

# dict of weights
n_classes = len(np.unique(y))
weight_0 = len(y) / (n_classes * y.value_counts()[0])
weight_1 = len(y) / (n_classes * y.value_counts()[1])
class_weights = {0: weight_0, 1: weight_1}

# use RandomizedSearchCV to find the best hyperparameter combination.
random_search = RandomizedSearchCV(GradientBoostingClassifier(),
                                   param_distributions=param,
                                   cv=5,
                                   n_iter=16,
                                   scoring='f1_macro',
                                   random_state=32,
                                   n_jobs=-1)
random_search.fit(X.values, y.values, sample_weight=sample_weights)

# print the best hyperparameter combination and its F-score (macro).
print("Best hyperparameter:", random_search.best_params_)
print(f"Best F-score (macro): {random_search.best_score_:.2%}")


/usr/local/lib/python3.10/site-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/site-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/site-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/site-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
 

Best hyperparameter: {'subsample': 0.8, 'n_estimators': 500, 'max_depth': 10, 'learning_rate': 0.05}
Best F-score (macro): 96.79%


In [80]:
duration = time.time()-t_start
h = '{:02d}'.format(int(duration//3600))
m = '{:02d}'.format(int((duration % 3600)//60))
s = '{:02d}'.format(int(duration % 60))

print('End:', datetime.datetime.now().strftime('%d-%m-%Y %H:%M:%S'))
print(f'Fine tuning execution time: {h}:{m}:{s}')

End: 09-05-2023 15:17:25
Fine tuning execution time: 01:35:28


# Final Model

In [81]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier

np.seterr(all='ignore')
sample_weights = compute_sample_weight(class_weight='balanced', y=y)
# model definition
xgb_model = GradientBoostingClassifier(max_depth=10, n_estimators=500, random_state=42, subsample=0.8)

# model training
xgb_model.fit(X.values, y.values, sample_weight=sample_weights)  
{'subsample': 0.8, 'n_estimators': 500, 'max_depth': 10, 'learning_rate': 0.05}
    

{'subsample': 0.8, 'n_estimators': 500, 'max_depth': 10, 'learning_rate': 0.05}

# Predict Kaggle Submission Data

In [150]:
df_test = pd.read_csv('../hachday_hotel-chain/dataset/test.csv', low_memory=False)
listID = df_test['id']


In [151]:
dft = df_test.copy()

In [152]:
# Rename 
dft = dft.rename(columns={'id':'id','Classificação do hotel': 'hotel_rating', 'Meses da reserva até o check-in': 'month_booking',
                        'Número de pernoites reservadas': 'number_stays_booking', 'Número de hospedes': 'number_guests',
                        'Regime de alimentação': 'meal_regime', 'Nacionalidade': 'nationality', 'Forma de Reserva': 'reservation_Form',
                        'Já se hospedou anterioremente': 'previously_hosted', 'Tipo do quarto reservado': 'type_booked_room',
                        'Reserva feita por agência de turismo': 'tourism_agency_booking', 'Reserva feita por empresa': 'company_booking',
                        'Reserva com Estacionamento': 'parking_Reservation', 'Reserva com Observações': 'reservation_Observations', 'Reserva Cancelada': 'booking_canceled'})



In [153]:
dft= dft.loc[:, ['id','hotel_rating','month_booking','nationality','parking_Reservation', 'reservation_Observations', 'number_stays_booking', 'reservation_Form']]

In [154]:
dft.head()

,id,hotel_rating,month_booking,nationality,parking_Reservation,reservation_Observations,number_stays_booking,reservation_Form
0,118345,4 estrelas,8,South Korea,Não,Nenhuma,13,Balcão
1,9500,5 estrelas,5,United States,Não,Nenhuma,4,Agência
2,34558,5 estrelas,4,United Kingdom,Não,Nenhuma,11,Agência
3,70816,4 estrelas,7,Spain,Não,Nenhuma,4,Agência
4,105321,4 estrelas,1,Italy,Não,1 a 3,8,B2B


## Deploy

In [156]:
transfomation_test( model = xgb_model, dft = dft, listID= listID, name = 'submission_v2' )

KeyError: "None of [Index(['number_guests'], dtype='object')] are in the [columns]"

# Resultado Kaggle

In [ ]:
result_kaggle = pd.DataFrame(columns = ['Versão','Algoritmo','f1_score_test','private_score', 'public_score'])

baseline = {'Versão':'baseline','Algoritmo':'regressionLogistic','private_score':0.669, 'public_score':0.669}
model_v1 = {'Versão': 'v1', 'Algoritmo':'XGBoost','f1_score_test': 0.971, 'private_score': 0.883, 'public_score': 0.881}
model_v2 = {'Versão': 'v2', 'Algoritmo':'XGBoost', 'f1_score_test': 0.968, 'private_score': 0.912, 'public_score': 0.911}


modelos = [baseline, model_v1, model_v2]
result_kaggle.append(modelos)

,Versão,Algoritmo,f1_score_test,private_score,public_score
0,baseline,regressionLogitic,NaN,0.669,0.669
1,v1,XBGoost,0.971,0.883,0.881
2,v2,XBGoost,0.968,0.912,0.911
